# 【申万】罗伯·瑞克超额现金流选股法则复现和延伸探讨


-- 本研报分析基于[【复现】系列十三:罗伯·瑞克超额现金流选股法则](https://www.joinquant.com/view/community/detail/30543ad72454c7648b03bae542af55c9?type=1)，感谢前人抛砖引玉，本文按照思路做了一些重新梳理

-- 更新部分，原分析采用了Tushare来获取部分财务数据，这里调整为全部采用聚宽数据。其中，本文调整了对企业自由现金流的计算方式，作为该现金流选股法的核心，其定义有待商榷，也请指正。最后，原分析采用了调取回测模块来进行计算，本文基于前序[致敬比乔斯基价值选股，对FFScore研报读后再整理](https://www.joinquant.com/view/community/detail/af5f2d96e4ab142bdd1701b14503fec9)的研究整理，将部分方法固化为共享函数，并采取因子化来进行回测验证。

## 理论整理

罗伯·瑞克，华尔街著名的资本大鳄，从事证券行业近 50 年。本篇介绍的罗伯·瑞克超额现金流选股法则是由其在接受股票书籍作者采访时，给投资者的投资建议所提炼而成。

### 罗伯·瑞克超额现金流选股模型

罗伯·瑞克投资策略从三个角度对投资标的进行考察

- 估值水平：股票具备合理的估值
- 分红水平：股票具备一定的分红收益
- 财务状况：公司的财务状况要求健康。 

以这三个角度出发，选择了 5 个指标，分别是<font color=Red>市净率、股息收益率、市盈率、借款总额占总资本比例、市现率</font>，共同构建了选股准则，形成了罗伯·瑞克超额现金流选股法则。其中，罗伯·瑞克尤其重视股价和超额现金流的关系，倾向于现金流相对市值更加充足的公司。

整理如下：

- 估值水平：用市净率、市盈率、股价/超额现金流来衡量股票的估值，要求买入的投资标的足够“便宜”，具有合理的估值。
- 分红水平：用股息收益率来衡量股票的分红水平，保证价值投资者能够获得一定的分红收益。
- 财务状况：用借款总额/总资本来衡量公司的偿债能力和财务结构，保证买入的投资标的具备足够强大的资产负债表。 

### 模型解构

基于罗伯·瑞克的思路和选择的指标，申万整理出的筛选逻辑，如下：

- 市净率低于 1.5 
- 市盈率低于市场平均值
- 股价/超额现金流量比低于 7.5
- 股利收益率高于市场平均值 
- 借款总额占总资本比例低于 33%

结合中国投资市场的实际情况之后，申万提出，在研究和回测中对部分原始标准进行调整： 

- 改为：市净率低于 3
- 改为：股价现金流量比低于全市场平均值的80%

本文以申万版本进行回测，同时，将沪深300作为市场的平均值，代入计算

### 指标定义和解读

这里仅讨论不能直接从因子库里获取的指标，以及对瑞克指标的思考分析：

- 超额现金流等价采用 <font color=Red>自由现金流（FCF）</font>

    做为模型的核心，自由现金流就是企业日常经营活动中产生的，减去各种税费，扣除企业相关投资需求后，能够分配给股东和债权人的最大现金流量。即经营活动产生的现金流量净额 - 股利 - 购买长期资产支出 + 出售长期资产得到的现金。
    
    如果一个企业的自由现金流越大，表明其内部产生现金的能力就越大，其可自由运用的内源资金也越多，企业对外融资的要求也越低，企业的财务状况越健康。此外，由于有真实的现金支出（收付实现制），自由现金流量不容易受会计方法的影响，也不受经理人的操纵，盈余管理（即受经理人操作空间更小的财务指标，但不能完全杜绝）的影响可以尽可能削弱。
    
    由于A股的核准制上市模式，相比较瑞克所在的美股，中国企业本身都受到一定的经营时间和业绩约束，具备良好的经营和业绩基础，因此上市后如果继续健康发展，理论上讲，使用自由现金流量的指标来评判是可行的，即该方法理论上是适应A股市场的。
    
    瑞克采用了股价相除，而股票价值本质上等于未来股利流入的现值，即未来现金流的折现。假设公司的自由现金流量增长量为零，不举债且将全部自由现金流量折现为股利分配给股东，即可以得出理论的股票价格等于平均自由现金流量与必要报酬率（以无风险收益为准）的比值。该比值可以理论上去衡量投资者的投资回报，即当自由现金流大于必要报酬率的倒数时，表明股票的市场价值被高估，反之，表明股票的市场价值被低估。所以，按照瑞克的分析方法，逻辑上是可以选出市场上被错误估价的股票，也就是达到要求买入的投资标的足够“便宜”，具有合理的估值。
    
    所以综上，瑞克认为决定股票内在价值是以股息收入为基础，以现金流贴现来操作，将股票的内在价值等同于其日后获得的全部股息的现值。那么超额现金流的逻辑其实就是在寻找投资对象的内在价值，即当市价低于这一内在稳固基础时，就出现买进的机会，相反，当市价高了内在价值时，就出现卖出的机会。在2022年大盘如此低迷的时期，讨论内在价值，有助于寻找更稳健长期的价值投资标的。

    聚宽中自由现金流因子是不能直接获取的，按照定义，整理得到：自由现金流 = 经营活动产生的现金流量净额 - 分配股利、利润或偿付利息支付的现金 - 购建固定资产、无形资产和其他长期资产支付的现金 + 处置固定资产、无形资产和其他长期资产收回的现金净额。
    
    $FCF = net\_operate\_cash\_flow - dividend\_interest\_payment - fix\_intan\_other\_asset\_acqui\_cash + fix\_intan\_other\_asset\_dispo\_cash$


- 借款占比（$debt\_ratio$）
    
    采用$\frac{长借+短借}{资产总计}$来计算，其中长期借款为$longterm\_loan$，短期借款为$shortterm\_loan$

### 模型因子化

- **估值水平**
    - 市净率：$pb<3$为1，否则为0
    - 市盈率：$pe\_ttm<pe\_ttm_{hs300}$为1，否则为0
    - 内在价值 = 股价/自由现金流：$intr\_value = \frac{close}{FCF}<\frac{close}{FCF_{hs300}}*0.8$为1，否则为0

- **分红水平**
    - 股息率：$dv\_ttm>dv\_ttm_{hs300}$为1，否则为0

- **财务状况**
    - 借款占比：$debt\_ratio=\frac{longterm\_loan + shortterm\_loan}{total\_assets}<0.33$为1，否则为0

根据上续讨论，对模型构建5因子模型，按总分排序，可以得到模型结果

$LScore = pb + pe + intr\_value + dv\_ttm + debt\_ratio$

## 申万对模型的评价

目前所测试的选股法则并不能算成熟的策略，仍有不足。其一、在 2008 年出现过长达 202 天，幅度为 71%的回撤；其二、策略相对基准的相对强弱波动较大，仅在 2008 年到 2010 年、2013 年后体现出明显的相对优势，而在其他 5 年间没有体现出稳定的相对优势；其三、策略持股个数波动略大，易造成交易执行的问题。当股票个数过少时，隐藏的风险和波动性加大。

申万提出三个简单的优化建议: 

- 吸收罗伯·瑞克超额现金流选股法则中的投资思维，在原有基础上扩展筛选股的考查纬度、或者调整具体的考核指标，进一步优化考察指标的参数
- 对选股数量和行业分布进行调整，利用分散化投资和行业中性的优势来避免策略的大幅波动，减少策略中明显不足的最大回撤指标；
- 将罗伯·瑞克超额现金流选股法则所选出的股票作为股票池，利用其它方法进一步筛选和判断入市时机。

In [2]:
class CashflowFactor(Factor):
    name = 'CashflowFactor'
    max_window = 1
    watch_date = None
    #
    dependencies = [
        # 估值水平
        'pb', 'pe_ttm', 
        'net_operate_cash_flow', 'dividend_interest_payment', 
        'fix_intan_other_asset_acqui_cash', 'fix_intan_other_asset_dispo_cash',
        'close',
        
        # 分红水平
        'dv_ttm',
        
        # 财务状况
        'longterm_loan', 'shortterm_loan', 'total_assets',
    ]
    #
    def calc(self, data: Dict) -> None:
        # 估值水平
        pb: pd.DataFrame = data['pb']
        pe_ttm: pd.DataFrame = data['pe_ttm']       
        fcf: pd.DataFrame = data['net_operate_cash_flow'] − data['dividend_interest_payment'] − data['fix_intan_other_asset_acqui_cash'] + data['fix_intan_other_asset_dispo_cash']
        intr_value: pd.DataFrame = data['close'] / fcf
            
        # 分红水平
        debt_ratio: pd.DataFrame = data['dv_ttm']
            
        # 财务状况        
        dv_ttm: pd.DataFrame = (data['longterm_loan'] + data['shortterm_loan']) / data['total_assets']    
        
        #
        indicator_tuple: Tuple = (pb, pe_ttm, fcf, intr_value, debt_ratio, dv_ttm)

        # 储存计算FFscore所需原始数据
        self.basic: pd.DataFrame = pd.concat(indicator_tuple).T.replace([-np.inf, np.inf], np.nan)
        #
        self.basic.columns = ['PB', 'PE_TTM', 'FCF', 'INTR_VALUE', 'DEBT_RATIO', 'DV_TTM']
#         self.fscore: pd.Series = self.basic.apply(sign).sum(axis=1)

SyntaxError: invalid character in identifier (<ipython-input-2-a31426d153c1>, line 24)